<a href="https://colab.research.google.com/github/anuraagkansara/SecureNLP/blob/master/code/T2_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
!git clone https://github.com/anuraagkansara/SecureNLP.git

fatal: destination path 'SecureNLP' already exists and is not an empty directory.


In [158]:
!pip install seqeval
!pip install transformers

In [0]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F

In [0]:
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

In [0]:
data_path = "SecureNLP/train/tokenized"


In [0]:

def sentenceRead(path, sep):
    total_sentences = 0
    filenames = os.listdir(path)                    # gets all the files in the folder where data is present
    sentences = []                                  # initialize empty list named sentence
    for filename in filenames:
        with open(path + filename) as file:                               
            lines = file.read()                     # reads entire file from a folder
        lines = lines.split(sep)                    # split the sentence by newline
        for line in lines:              
            l = [] 
            total_sentences += 1                                 # inilialize empty list to hold the single lines
            for i in line.splitlines():             # for x in a single line of a sentence
                # if len(i) < 1:
                #   continue
                w = i.split(' ')                    # split x into two parts...... one having the word and another having the iob tag
                if w[0] == "''":                    # if we found no word on the line of a sentence, then make w[0] as "
                    w[0] = '"'                      # the data is such that after every sentence (full stop),
                if len(w) == 2:                     # we have only a newline character in a line, followed by a word in next line 
                    k = []
                    k.append(total_sentences)
                    k.append(w[0])
                    k.append(w[1])
                    sentences.append(k)           # only when we have a 'word' and a 'iob' tag in w (making len(w) == 2), we will append

            # if sum([x[1] == 'O' for x in l]) < len(l): # number of 'Os' found are less than length of the sentence, then add that sentence
            #  sentences.append(l)                         # to the list, since number of 'Os' are less means we have other iob tags too.
    return sentences,total_sentences                         # append  only those sentences which have tags other than 'O' as well


In [0]:
train,train_count = sentenceRead("SecureNLP/train/tokenized/", " \n")   #training data

In [0]:
dev,dev_count = sentenceRead("SecureNLP/dev/tokenized/", " \n")
test = []
test_count = 0
for i in [1, 2, 3]:
    x, tc = sentenceRead(f"SecureNLP/test_{i}/tokenized/", "\n\n")
    test_count += tc 
    test.extend(x)  #adds another list to end of previous list
train.extend(test)
train.extend(dev)

In [0]:
df_data = pd.DataFrame(train, columns = ["Sentence #",'Word', 'Tag']) 
df_data.insert(2,"POS","None")

In [0]:
# Working with testing data

total_sentences = 0
sentencesout = list()
# tokenout = []
with open('SecureNLP/Task2.test') as fileout:                               
  linesout = fileout.read()  
linesout = linesout.split('\n\t\n')
# print(linesout[0])
for line in linesout:
  
  l = []     
  t=[]                             # inilialize empty list to hold the single lines
  total_sentences += 1
  for i in line.splitlines():             # for x in a single line of a sentence
      # if len(i) < 1:
      #   continue
      w = i.split('\t')                    # split x into two parts...... one having the word and another having the iob tag
      if w[0] == "''":                    # if we found no word on the line of a sentence, then make w[0] as "
          w[0] = '"'                      # the data is such that after every sentence (full stop),
      if len(w) == 2:                     # we have only a newline character in a line, followed by a word in next line
        k = []
        k.append(total_sentences)
        k.append(w[0])
        k.append(w[1])
        sentencesout.append(k)           # only when we have a 'word' and a 'iob' tag in w (making len(w) == 2), we will append
  # t = np.array(t)                       # converts entire list 'l' into numpy array
  # if sum([x[1] == 'O' for x in l]) < len(l): # number of 'Os' found are less than length of the sentence, then add that sentence 
  # tokenout.append(t) 

df_test = pd.DataFrame(sentencesout, columns = ['Sentence #','Word', 'Tag']) 
df_test.insert(2,"POS","None")
# sen_no_test = []
# num = 1
# for i in df_test['Word']:
#   sen_no_test.append("Sentence: "+str(num))
#   if i == '.':
#     num += 1
# df_test.insert(0,"Sentence #",sen_no_test)

In [243]:
# df_test = df_test[:-1]
df_test.tail(5)
df_data[df_data['Word'] == 'Alliance']
# df_test[df_test['Sentence #'] == 'Sentence: 99']
# df_test

,Sentence #,Word,POS,Tag
194,10,Alliance,None,O
221,10,Alliance,None,I-Entity
171744,6089,Alliance,None,O


In [244]:
df_data.Tag.unique()

array(['O', 'B-Entity', 'I-Entity', 'B-Action', 'B-Modifier', 'I-Action',
       'I-Modifier'], dtype=object)

In [245]:
df_data.Tag.value_counts()


O             332239
I-Entity       23333
B-Entity        8503
B-Action        3990
B-Modifier      2498
I-Action         824
I-Modifier       138
Name: Tag, dtype: int64

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        # print(self.data)
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
        

    def get_next(self):
        try:
            s = self.grouped["{}".format(self.n_sent)]
            self.n_sent += 1
            print("inside getnext")
            return s
        except:
            return None

In [0]:
getter = SentenceGetter(df_data)

In [0]:
getter_test = SentenceGetter(df_test)

In [0]:
# df_data.head(50)

In [0]:
sentences = [[s[0] for s in sent] for sent in getter.sentences]
# sentences[:5]

In [251]:
sentences_test = [[s[0] for s in sent] for sent in getter_test.sentences]
# sentences_test[:4]
len(sentences_test)

618

In [252]:
poses = [[s[1] for s in sent] for sent in getter.sentences]
print(poses[0])
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0])

['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',

In [253]:
poses_test = [[s[1] for s in sent] for sent in getter_test.sentences]
print(poses_test[0])
labels_test = [[s[2] for s in sent] for sent in getter_test.sentences]
print(labels_test[0])

['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [0]:
tags_vals = list(set(df_data["Tag"].values))


In [0]:
tags_vals_test = list(set(df_test["Tag"].values))


In [0]:
tags_vals.append('X')
tags_vals.append('[CLS]')
tags_vals.append('[SEP]')
tags_vals = set(tags_vals)


In [0]:
tags_vals_test.append('X')
tags_vals_test.append('[CLS]')
tags_vals_test.append('[SEP]')
tags_vals_test = set(tags_vals_test)


In [258]:
df_data.Tag.value_counts()

O             332239
I-Entity       23333
B-Entity        8503
B-Action        3990
B-Modifier      2498
I-Action         824
I-Modifier       138
Name: Tag, dtype: int64

In [259]:
df_test.Tag.value_counts()

O             12119
I-Entity        499
B-Entity        249
B-Action        125
B-Modifier       79
I-Action          9
Name: Tag, dtype: int64

In [0]:
# Set a dict for mapping id to tag name
#tag2idx = {t: i for i, t in enumerate(tags_vals)}

# Recommend to set it by manual define, good for reusing
tag2idx={'I-Entity': 1,
 'B-Entity': 2,
 'B-Action': 0,
 'B-Modifier': 3,
 'I-Action': 4,
 'I-Modifier': 5,
 'O' : 6,
 'X' : 9,
 '[CLS]':7,
 '[SEP]':8}

In [261]:
tag2idx

{'B-Action': 0,
 'B-Entity': 2,
 'B-Modifier': 3,
 'I-Action': 4,
 'I-Entity': 1,
 'I-Modifier': 5,
 'O': 6,
 'X': 9,
 '[CLS]': 7,
 '[SEP]': 8}

In [262]:
# Mapping index to name
tag2name={tag2idx[key] : key for key in tag2idx.keys()}
tag2name

{0: 'B-Action',
 1: 'I-Entity',
 2: 'B-Entity',
 3: 'B-Modifier',
 4: 'I-Action',
 5: 'I-Modifier',
 6: 'O',
 7: '[CLS]',
 8: '[SEP]',
 9: 'X'}

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [0]:
vocabulary = 'drive/My Drive/NLP project/vocab.txt'

In [0]:
# Len of the sentence must be not bigger than the training model
# See model's 'max_position_embeddings' = 512
max_len  = 45

In [0]:
# load tokenizer, with manual file address or pretrained address
tokenizer=BertTokenizer(vocab_file=vocabulary,do_lower_case=False)

In [267]:
tokenized_texts = []
word_piece_labels = []
i_inc = 0
for word_list,label in (zip(sentences,labels)):
    temp_lable = []
    temp_token = []
    
    # Add [CLS] at the front 
    temp_lable.append('[CLS]')
    temp_token.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token.append(token)
            if m==0:
                temp_lable.append(lab)
            else:
                temp_lable.append('X')  
                
    # Add [SEP] at the end
    temp_lable.append('[SEP]')
    temp_token.append('[SEP]')
    
    tokenized_texts.append(temp_token)
    word_piece_labels.append(temp_lable)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token)))
        print("texts:%s"%(" ".join(temp_token)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable)))
        print("lables:%s"%(" ".join(temp_lable)))
    i_inc +=1

No.0,len:200
texts:[CLS] Post ##ed on October 9 , 2014 by Steven Ada ##ir Over the last few months , Vol ##ex ##ity has been tracking a particularly remarkable advanced persistent threat ( AP ##T ) operation involving strategic web compromise ##s of websites in Hong Kong and Japan . The experts of G D ##AT ##A ' s Security ##L ##ab ##s discovered a c ##y ##ber - espionage campaign that perfectly ex ##em ##p ##li ##fies the way how targeted attacks work . E ##qua ##tion ##D ##rug is one of the main espionage platforms used by the E ##qua ##tion Group ] , a highly sophisticated threat actor that has been engaged in multiple C ##NE ( computer network exploitation ) operations dating back to 2001 , and perhaps as early as 1996 . Over the last few months , the Crow ##d ##S ##tri ##ke Intelligence team has been tracking a campaign of highly targeted events focused on entities in the U . S . A c ##y ##ber ##es ##pion ##age campaign against a range of targets , mainly in the energy sector , ga

In [268]:
tokenized_texts_test = []
word_piece_labels_test = []
i_inc = 0
for word_list,label in (zip(sentences_test,labels_test)):
    temp_lable_test = []
    temp_token_test = []
    
    # Add [CLS] at the front 
    temp_lable_test.append('[CLS]')
    temp_token_test.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token_test.append(token)
            if m==0:
                temp_lable_test.append(lab)
            else:
                temp_lable_test.append('X')  
                
    # Add [SEP] at the end
    temp_lable_test.append('[SEP]')
    temp_token_test.append('[SEP]')
    
    tokenized_texts_test.append(temp_token_test)
    word_piece_labels_test.append(temp_lable_test)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token_test)))
        print("texts:%s"%(" ".join(temp_token_test)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable_test)))
        print("lables:%s"%(" ".join(temp_lable_test)))
    i_inc +=1

No.0,len:18
texts:[CLS] Low - profile information - stealing Trojan is used only against high - value targets . [SEP]
No.0,len:18
lables:[CLS] O O O O O O O O O O O O X O O O [SEP]
No.1,len:34
texts:[CLS] S ##yman ##tec has uncovered an el ##usive Trojan used by the c ##y ##ber ##es ##pion ##age group behind the € œ ##D ##uke € family of ma ##l ##ware . [SEP]
No.1,len:34
lables:[CLS] O X X O O O O X O O O O O X X X X X O O O O O X X O O O O X X O [SEP]
No.2,len:44
texts:[CLS] Sea ##du ##ke ( detected by S ##yman ##tec as Trojan . Sea ##du ##ke ) is a low - profile information - stealing Trojan which appears to be reserved for attacks against a small number of high - value targets . [SEP]
No.2,len:44
lables:[CLS] O X X O O O O X X O O O O X X O O O O O O O X O O O O O O O O O O O O O O O O O O O [SEP]
No.3,len:21
texts:[CLS] Sea ##du ##ke has been used in attacks against a number of major , government - level targets . [SEP]
No.3,len:21
lables:[CLS] O X X O O O O O O O O O O O O O O O O

In [269]:
# Make text token into id
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[  101  3799  1174  1113  1357   130   117  1387  1118  6536 19222  3161
  3278  1103  1314  1374  1808   117  5713 11708  1785  1144  1151 10066
   170  2521  9495  3682 15970  4433   113 10997  1942   114  2805  5336
  7061  5127 13018  1116  1104 12045  1107  3475  3462]


In [270]:
# Make text token into id
input_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_test[0])

[  101  8274   118  6168  1869   118 11569 28099  1110  1215  1178  1222
  1344   118  2860  7539   119   102     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0]


In [271]:
# Make label into id, pad with "O" meaning others
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=max_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
print(tags[0])

[7 6 9 6 6 6 6 6 6 6 6 9 6 6 6 6 6 6 6 9 9 6 6 6 6 6 6 6 6 6 6 6 9 6 6 6 6
 6 6 9 6 6 6 6 6]


In [272]:
# Make label into id, pad with "O" meaning others
tags_test = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels_test],
                     maxlen=max_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
print(tags_test[0])

[7 6 6 6 6 6 6 6 6 6 6 6 6 9 6 6 6 8 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6]


In [0]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
# attention_masks[0]

In [0]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_test = [[int(i>0) for i in ii] for ii in input_ids_test]
# attention_masks_test[0]

In [0]:
# Since only one sentence, all the segment set to 0
segment_ids = [[0] * len(input_id) for input_id in input_ids]
# segment_ids[0]

In [0]:
# Since only one sentence, all the segment set to 0
segment_ids_test = [[0] * len(input_id) for input_id in input_ids_test]
# segment_ids[0]

In [0]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=4, test_size=0.30)

In [278]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(6619, 2838, 6619, 2838)

In [279]:
len(input_ids_test)
len(tags_test)
len(attention_masks_test)
len(segment_ids_test)

618

In [0]:
# print(input_ids)
# print()
# print(val_inputs)
# print()
# print(val_masks)
# print()
# print(val_tags)

test_inputs = torch.tensor(input_ids_test)
test_tags = torch.tensor(tags_test)
test_masks = torch.tensor(attention_masks_test)
test_segs = torch.tensor(segment_ids_test)

In [0]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [0]:
batch_num = 32

In [0]:
# Only set token embedding, attention embedding, no segment embedding
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [0]:
# Only set token embedding, attention embedding, no segment embedding

test_data = TensorDataset(test_inputs, test_masks, test_tags)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=valid_sampler, batch_size=batch_num)

In [0]:
model_file_address = 'drive/My Drive/NLP project'

In [0]:
# Will load config and weight with from_pretrained()
model = BertForTokenClassification.from_pretrained(model_file_address,num_labels=len(tag2idx))

In [0]:
model;

In [0]:
# Set model to GPU,if you are using GPU machine
model.cuda();

In [0]:
# Set epoch and grad max num
epochs = 5
max_grad_norm = 1.0

In [0]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [0]:
# True: fine tuning all the layers 
# False: only fine tuning the classifier layers
FULL_FINETUNING = True

In [0]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [0]:
# TRAIN loop
model.train();

In [311]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 6619
  Batch size = 32
  Num steps = 1035


Epoch:  20%|██        | 1/5 [01:07<04:30, 67.52s/it]

Train loss: 0.00844154477564861


Epoch:  40%|████      | 2/5 [02:15<03:23, 67.77s/it]

...

Train loss: 0.008513737697399488


Epoch:  80%|████████  | 4/5 [04:34<01:08, 68.58s/it]

Train loss: 0.008517878119252253


Epoch: 100%|██████████| 5/5 [05:44<00:00, 68.85s/it]

Train loss: 0.008483695965215275


In [0]:
bert_out_address = 'drive/My Drive/NLP project'

In [0]:
# Make dir if not exits
if not os.path.exists(bert_out_address):
        os.makedirs(bert_out_address)

In [0]:
# Save a trained model, configuration and tokenizer
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [0]:
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")

In [316]:
# Save model into file
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

('drive/My Drive/NLP project/vocab.txt',)

In [0]:
model = BertForTokenClassification.from_pretrained(bert_out_address,num_labels=len(tag2idx))

In [0]:
# Set model to GPU
model.cuda();

In [0]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [0]:
# Evalue loop
model.eval();

In [321]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):

    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)



print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

# Save the report into file
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =2838
  Batch size = 32
f1 socre: 0.459845
Accuracy score: 0.890770
***** Eval results *****

           precision    recall  f1-score   support

   Entity     0.3645    0.4638    0.4082      1973
 Modifier     0.5230    0.6263    0.5700       562
   Action     0.5067    0.6227    0.5587       917

micro avg     0.4047    0.5324    0.4598      3452
macro avg     0.4281    0.5324    0.4745      3452

f1 socre: 0.459845
Accuracy score: 0.890770


In [322]:
type(test_dataloader)
type(train_dataloader)

torch.utils.data.dataloader.DataLoader

In [323]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true_test = []
y_pred_test = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(test_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(test_dataloader):
    print(step)
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch

#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true_test.append(temp_1)
        y_pred_test.append(temp_2)
    if step == 18:
      break


print("f1 socre: %f"%(f1_score(y_true_test, y_pred_test)))
print("Accuracy score: %f"%(accuracy_score(y_true_test, y_pred_test)))

# Get acc , recall, F1 result report
report = classification_report(y_true_test, y_pred_test,digits=4)

# Save the report into file
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true_test, y_pred_test)))
    print("Accuracy score: %f"%(accuracy_score(y_true_test, y_pred_test)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true_test, y_pred_test)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true_test, y_pred_test)))
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =618
  Batch size = 32
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
f1 socre: 0.346362
Accuracy score: 0.841175
***** Eval results *****

           precision    recall  f1-score   support

   Entity     0.2054    0.5812    0.3036       234
   Action     0.3023    0.7845    0.4365       116
 Modifier     0.3408    0.8133    0.4803        75

micro avg     0.2326    0.6776    0.3464       425
macro avg     0.2558    0.6776    0.3710       425

f1 socre: 0.346362
Accuracy score: 0.841175


In [324]:
print(y_pred_test)
print()
print(y_true_test)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'X', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Entity', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', 'O'], ['B-Entity', 'I-Entity', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Action', 'O', 'B-Action', 'B-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'I-Entity', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O

In [325]:
sentences_test[0]

['Low',
 '-',
 'profile',
 'information',
 '-',
 'stealing',
 'Trojan',
 'is',
 'used',
 'only',
 'against',
 'high-',
 'value',
 'targets',
 '.']

In [326]:
outputPred = open('Task2.out','w+')
ols = []
print(len(sentences_test))
print(len(y_pred_test))
print((sentences_test[617]))
for word,label in zip(sentences_test,y_pred_test):
  # print(len(word),'\n',len(label))
  # print()
  if len(word)>len(label):
      # print(word)
      # print("Word Length: ",len(word))
      # print("label Length: ",len(label))
    for i in range(len(label)):
      ols.append(word[i]+'\t'+label[i]+'\n')
    for i in range(len(label),len(word)):
      ols.append(word[i]+'\t'+'O'+'\n')
  else:
    for i in range(len(word)):
      ols.append(word[i]+'\t'+label[i]+'\n')
      # print(word[i]+'\t'+label[i]+'\n')
  ols.append('\n')

for word in sentences_test[608:]:
  for i in word:
    ols.append(i+'\t'+'O'+'\n')
  ols.append('\n')

print(ols)
# print('\n\n')
# ols = ols[:-2]
print(len(ols))
ols = ''.join(ols)
# print(len(ols))
outputPred.write(ols)

618
608
['In', 'the', 'meantime', ',', 'register', 'now', 'for', 'the', 'April', '1st', 'CrowdCast', '.']
['Low\tO\n', '-\tO\n', 'profile\tO\n', 'information\tO\n', '-\tO\n', 'stealing\tO\n', 'Trojan\tO\n', 'is\tO\n', 'used\tO\n', 'only\tO\n', 'against\tO\n', 'high-\tO\n', 'value\tO\n', 'targets\tO\n', '.\tO\n', '\n', 'Symantec\tO\n', 'has\tO\n', 'uncovered\tO\n', 'an\tO\n', 'elusive\tO\n', 'Trojan\tO\n', 'used\tO\n', 'by\tO\n', 'the\tO\n', 'cyberespionage\tO\n', 'group\tO\n', 'behind\tO\n', 'the\tO\n', '€\tO\n', 'œDuke\tO\n', '€\tO\n', '\u200b\tO\n', 'family\tO\n', 'of\tO\n', 'malware\tO\n', '.\tO\n', '\n', 'Seaduke\tO\n', '(\tO\n', 'detected\tO\n', 'by\tO\n', 'Symantec\tO\n', 'as\tO\n', 'Trojan\tO\n', '.\tX\n', 'Seaduke\tX\n', ')\tO\n', 'is\tO\n', 'a\tO\n', 'low\tO\n', '-\tO\n', 'profile\tO\n', 'information-\tI-Entity\n', 'stealing\tO\n', 'Trojan\tO\n', 'which\tO\n', 'appears\tO\n', 'to\tO\n', 'be\tO\n', 'reserved\tO\n', 'for\tO\n', 'attacks\tO\n', 'against\tO\n', 'a\tO\n', 'small\tO

115591

In [327]:
df_test

,Sentence #,Word,POS,Tag
0,1,Low,None,O
1,1,-,None,O
2,1,profile,None,O
3,1,information,None,O
4,1,-,None,O
...,...,...,...,...
13075,618,the,None,O
13076,618,April,None,O
13077,618,1st,None,O
13078,618,CrowdCast,None,O
